In [ ]:
# import modules and define functions
from pyscf import gto, scf, dft
import pandas as pd
import numpy as np
import os
os.environ['OMP_NUM_THREADS'] = "1"
def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

# Functional dependences of SIE

In [ ]:
# calculate SIE for DFT
# build H2
xc_sie_dict = {}
atom = 'H 0 0 0' #; H 0 0 0.74'
basis = 
h_0_2 = gto.Mole()
h_0_2.build(atom=atom,basis=basis,charge=0,spin=1)
h_0_2_uhf = scf.UHF(h_0_2).run()
j, k = h_0_2_uhf.get_jk()
rho = h_0_2_uhf.make_rdm1()
mo_coeff_uhf = h_0_2_uhf.mo_coeff
mo_occ_uhf = h_0_2_uhf.mo_occ
J = np.dot(rho[0],j[0])[mo_occ_uhf[0]>0]
K = np.dot(rho[0],k[0])[mo_occ_uhf[0]>0]
sie = J[0][0] - K[0][0]
xc_sie_dict['HF'] = ['H', 0, sie]
# obtain SIE for various xc
xcs_lda = []
xcs_gga = []
xcs_meta = []
xcs = xcs_lda + xcs_gga + xcs_meta
for xc in xcs:
    h_0_2_dft = dft.UKS(h_0_2, xc=xc).run()
    dft_scf_dict = h_0_2_dft.scf_summary
    coul = dft_scf_dict['coul']
    exc = dft_scf_dict['exc']
    sie = coul + exc
    xc_sie_dict[xc] = ['H', 0, sie]

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame(xc_sie_dict).T
df.columns = ['Molecule', 'Theoretical', 'Found']
print(df)

# Functional dependences of IP

In [ ]:
# build CO with minimal basis
basis = 
xcs_lda = []
xcs_gga = []
xcs_meta = []
xcs = xcs_lda + xcs_gga + xcs_meta
xc_ip_dict = {}
for xc in xcs:
    n2_0_1 = gto.Mole().build(atom='N 0 0 0; N 0 0 1.098',basis=basis,charge=0,spin=0)
    n2_1_2 = gto.Mole().build(atom='N 0 0 0; N 0 0 1.098',basis=basis,charge=1,spin=1)
    n2_0_1_dft = dft.DFT(n2_0_1, xc=xc).run()
    n2_1_2_dft = dft.DFT(n2_1_2, xc=xc).run()
    s2_n2 = n2_1_2_dft.spin_square()[0]
    mo_occ = n2_0_1_dft.mo_occ
    mo_energy = n2_0_1_dft.mo_energy
    e_homo_n2 = mo_energy[mo_occ>0][-1]
    IP_N2 = n2_1_2_dft.e_tot - n2_0_1_dft.e_tot
    xc_ip_dict[xc] = ['N\u2082', 0.573, IP_N2, s2_n2]

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame(xc_ip_dict).T
df.columns = ['Molecule', 'Expt', 'IP_calc', '<S^2>']
print(df)